In [1]:
## Bring in needed mods
import glob

In [2]:
## Set file to drop
## We do not have the matching file for this samples
drop_file = '../FASTQ/JW-S1_S1_L001_R2_001.fastq.gz' 

In [3]:
## Load in fastq file names
file_paths_r1 = sorted(glob.glob('../FASTQ/*R1*.fastq.gz'))
file_paths_r2 = sorted([a for a in glob.glob('../FASTQ/*R2*.fastq.gz') if a != drop_file])

In [4]:
## Set referance path
refpath = '../REFERENCE/XL280_NP_NUCLEAR_FINAL_July2020.fasta'

In [5]:
%%bash

## Index the fasta file
bwa index ../REFERENCE/XL280_NP_NUCLEAR_FINAL_July2020.fasta

[bwa_index] Pack FASTA... 0.11 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 9.23 seconds elapse.
[bwa_index] Update BWT... 0.10 sec
[bwa_index] Pack forward-only FASTA... 0.06 sec
[bwa_index] Construct SA from BWT and Occ... 4.04 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index ../REFERENCE/XL280_NP_NUCLEAR_FINAL_July2020.fasta
[main] Real time: 14.893 sec; CPU: 13.554 sec


In [6]:
## Initate line
lines = []

## Iterate thru R1 path
for f1 in file_paths_r1:
    
    ## Set path to R2
    f2 = 'R2'.join(f1.split('R1'))
    
    ## Check work
    assert f2 in file_paths_r2, "Missing fastq file: %s"%f2
    
    ## Set the out bam name and path
    outbam =  '../BAMS/'+f1.split('/')[-1].split('R1')[0]+'.bam'
    
    ## Write a the bwa command
    bwaline = 'bwa mem -a -M %s %s %s | samtools view -b | samtools sort -o %s\nsamtools index %s\n\n'%(refpath,f1,f2,outbam,outbam)

    ## Append the bwa line
    lines.append(bwaline)
    
## Open file for writting bwa file too
fout = open('./bwa_XL280np.sh','w')
    
## Write to file, and close the file
fout.writelines(lines);fout.close()

In [ ]:
%%bash

## Make the bams dir
mkdir -p ../BAMS

## Run our bwa commands
bash bwa_XL280np.sh

In [11]:
## Gather the bam files
bams = sorted(glob.glob('../BAMS/*.bam'))

## Print the length
len(bams)

3

In [12]:
## Make a list of the bam files for analysis
## Initate list
depthlines = []

## Iterate thrut he bam files and append
for b in bams:
    depthlines.append(b+'\n')

## Open depth file to write
depthfile = open('../DATA/bamlist.txt','w')
    
## Write the lines then close
depthfile.writelines(depthlines);depthfile.close();

In [13]:
%%bash

## Call samtools depth ftn
samtools depth -f ../DATA/bamlist.txt > ../DATA/SampleDepths.tsv

In [14]:
%%bash

## Gzip the file
gzip ../DATA/SampleDepths.tsv